In [1]:
import tensorflow as tf
from tensorflow import keras
import os
from densenet_models import create_densenet169_model, create_densenet169_model_opt, create_densenet169_model_opt_weights
import wandb
from wandb.keras import WandbCallback

2024-05-23 09:08:07.394698: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9360] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-23 09:08:07.394747: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-23 09:08:07.394791: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1537] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-23 09:08:07.405930: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [17]:
wandb.login(key="0f45347e0cafc15c3aafb216ff27852b72c63c56")


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/katril18/.netrc


True

In [18]:
wandb.init(project="katrmaster")

wandb: Currently logged in as: katrinalie (katrinamaster). Use `wandb login --relogin` to force relogin


In [2]:
train_ds = keras.utils.image_dataset_from_directory(
    directory="../../dataset/global_15k/train",
    labels='inferred',
    label_mode='categorical',
    shuffle=True,
    batch_size=128,
    image_size=(224, 224)
    )
val_ds = keras.utils.image_dataset_from_directory(
    directory="../../dataset/global_15k/val",
    labels='inferred',
    label_mode='categorical',
    shuffle=True,
    batch_size=128,
    image_size=(224, 224)
    )

Found 312645 files belonging to 73 classes.


2024-05-20 14:40:50.357208: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1883] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31127 MB memory:  -> device: 0, name: Tesla V100-SXM3-32GB, pci bus id: 0000:5c:00.0, compute capability: 7.0


Found 50568 files belonging to 73 classes.


In [2]:
train_ds = keras.utils.image_dataset_from_directory(
    directory="../../../organized_dataset_v7/dataset/train",
    labels='inferred',
    label_mode='categorical',
    shuffle=True,
    batch_size=64,
    image_size=(224, 224)
    )
val_ds = keras.utils.image_dataset_from_directory(
    directory="../../../organized_dataset_v7/dataset/val",
    labels='inferred',
    label_mode='categorical',
    shuffle=True,
    batch_size=64,
    image_size=(224, 224)
    )

Found 430968 files belonging to 12 classes.


2024-05-23 09:08:50.964332: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1883] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31125 MB memory:  -> device: 0, name: Tesla V100-SXM3-32GB, pci bus id: 0000:5c:00.0, compute capability: 7.0


Found 37219 files belonging to 12 classes.


In [25]:
ls

densenet_fine_tuning.ipynb  densenet_models.py  wandb/
densenet.ipynb              __pycache__/


# DenseNet169

In [ ]:
# initialize wandb
wandb.init(project="DenseNet", config={"learning_rate": 0.001, "epochs": 40, "verbose": 1,"name": "DenseNet169", "architecture": "DenseNet169"})

# configs
cfg = wandb.config

# create the model
model = create_densenet169_model(len(train_ds.class_names))

In [4]:
# list of callbacks
callbacks = [
            WandbCallback(mode="min", monitor="val_loss", save_graph=True),
            tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=3, min_lr=0.00001),
            tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5),
          ]

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


In [ ]:
model.fit(train_ds, epochs=cfg.epochs, batch_size=32, verbose=cfg.verbose, validation_data=val_ds, callbacks=callbacks) # train the model

In [8]:
wandb.finish()

## DenseNet169 Optimized

In [3]:
# initialize wandb
wandb.init(project="DenseNet", config={"learning_rate": 0.001, "epochs": 40, "verbose": 1,"name": "DenseNet169Opt", "architecture": "DenseNet169Opt"})

# configs
cfg = wandb.config

# create the model
model = create_densenet169_model_opt_weights(len(train_ds.class_names))

wandb: Currently logged in as: katrinalie (katrinamaster). Use `wandb login --relogin` to force relogin


In [5]:
model.fit(train_ds, epochs=cfg.epochs, batch_size=128, verbose=cfg.verbose, validation_data=val_ds, callbacks=callbacks) # train the model

Epoch 1/40


2024-05-23 09:09:38.753820: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8907
2024-05-23 09:09:41.755911: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fb311689be0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-05-23 09:09:41.755951: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Tesla V100-SXM3-32GB, Compute Capability 7.0
2024-05-23 09:09:41.762910: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-05-23 09:09:41.869742: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


6734/6734 [==============================] - ETA: 0s - loss: 0.9998 - accuracy: 0.6549INFO:tensorflow:Assets written to: /home/katril18/DOC_prev_code/plankton_classification/models/densenet/wandb/run-20240523_090910-2yduig3g/files/model-best/assets


INFO:tensorflow:Assets written to: /home/katril18/DOC_prev_code/plankton_classification/models/densenet/wandb/run-20240523_090910-2yduig3g/files/model-best/assets
wandb: Adding directory to artifact (/home/katril18/DOC_prev_code/plankton_classification/models/densenet/wandb/run-20240523_090910-2yduig3g/files/model-best)... Done. 0.3s


6734/6734 [==============================] - 793s 116ms/step - loss: 0.9998 - accuracy: 0.6549 - val_loss: 0.4490 - val_accuracy: 0.8502 - lr: 0.0010
Epoch 2/40
6734/6734 [==============================] - ETA: 0s - loss: 0.7713 - accuracy: 0.7358INFO:tensorflow:Assets written to: /home/katril18/DOC_prev_code/plankton_classification/models/densenet/wandb/run-20240523_090910-2yduig3g/files/model-best/assets


...

6734/6734 [==============================] - 752s 112ms/step - loss: 0.4147 - accuracy: 0.8566 - val_loss: 0.2285 - val_accuracy: 0.9281 - lr: 1.0000e-04
Epoch 29/40
6734/6734 [==============================] - 704s 105ms/step - loss: 0.4127 - accuracy: 0.8573 - val_loss: 0.2332 - val_accuracy: 0.9283 - lr: 1.0000e-04
Epoch 30/40
6734/6734 [==============================] - ETA: 0s - loss: 0.4113 - accuracy: 0.8575INFO:tensorflow:Assets written to: /home/katril18/DOC_prev_code/plankton_classification/models/densenet/wandb/run-20240523_090910-2yduig3g/files/model-best/assets


INFO:tensorflow:Assets written to: /home/katril18/DOC_prev_code/plankton_classification/models/densenet/wandb/run-20240523_090910-2yduig3g/files/model-best/assets
wandb: Adding directory to artifact (/home/katril18/DOC_prev_code/plankton_classification/models/densenet/wandb/run-20240523_090910-2yduig3g/files/model-best)... Done. 0.3s


6734/6734 [==============================] - 746s 111ms/step - loss: 0.4113 - accuracy: 0.8575 - val_loss: 0.2273 - val_accuracy: 0.9288 - lr: 1.0000e-04
Epoch 31/40
6734/6734 [==============================] - 702s 104ms/step - loss: 0.4102 - accuracy: 0.8583 - val_loss: 0.2322 - val_accuracy: 0.9287 - lr: 1.0000e-04
Epoch 32/40
6734/6734 [==============================] - 706s 105ms/step - loss: 0.4086 - accuracy: 0.8595 - val_loss: 0.2297 - val_accuracy: 0.9294 - lr: 1.0000e-04
Epoch 33/40
6734/6734 [==============================] - 704s 105ms/step - loss: 0.4069 - accuracy: 0.8593 - val_loss: 0.2274 - val_accuracy: 0.9302 - lr: 1.0000e-04
Epoch 34/40
6734/6734 [==============================] - ETA: 0s - loss: 0.4027 - accuracy: 0.8610INFO:tensorflow:Assets written to: /home/katril18/DOC_prev_code/plankton_classification/models/densenet/wandb/run-20240523_090910-2yduig3g/files/model-best/assets


INFO:tensorflow:Assets written to: /home/katril18/DOC_prev_code/plankton_classification/models/densenet/wandb/run-20240523_090910-2yduig3g/files/model-best/assets
wandb: Adding directory to artifact (/home/katril18/DOC_prev_code/plankton_classification/models/densenet/wandb/run-20240523_090910-2yduig3g/files/model-best)... Done. 0.3s


6734/6734 [==============================] - 750s 111ms/step - loss: 0.4027 - accuracy: 0.8610 - val_loss: 0.2238 - val_accuracy: 0.9299 - lr: 1.0000e-05
Epoch 35/40
6734/6734 [==============================] - 707s 105ms/step - loss: 0.4021 - accuracy: 0.8609 - val_loss: 0.2281 - val_accuracy: 0.9292 - lr: 1.0000e-05
Epoch 36/40
6734/6734 [==============================] - 693s 103ms/step - loss: 0.4007 - accuracy: 0.8611 - val_loss: 0.2275 - val_accuracy: 0.9294 - lr: 1.0000e-05
Epoch 37/40
6734/6734 [==============================] - 707s 105ms/step - loss: 0.4029 - accuracy: 0.8605 - val_loss: 0.2242 - val_accuracy: 0.9297 - lr: 1.0000e-05
Epoch 38/40
6734/6734 [==============================] - 735s 109ms/step - loss: 0.4019 - accuracy: 0.8609 - val_loss: 0.2256 - val_accuracy: 0.9293 - lr: 1.0000e-05
Epoch 39/40
6734/6734 [==============================] - 703s 104ms/step - loss: 0.4019 - accuracy: 0.8604 - val_loss: 0.2269 - val_accuracy: 0.9295 - lr: 1.0000e-05


In [6]:
wandb.finish()

accuracy,▁▄▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇███████████████
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▅▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▃▄▅▅▆▆▆▆▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇███████████████
val_loss,█▆▅▄▄▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,0.8604
best_epoch,33
best_val_loss,0.22379
epoch,38
loss,0.40188
val_accuracy,0.92947
